In [2]:
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### The TensorFlow Dataset framework – main components

The TensorFlow Dataset framework has two main components:

* The Dataset
* An associated Iterator

The Dataset is basically where the data resides. This data can be loaded in from a number of sources – existing tensors, numpy arrays and numpy files, the `TFRecord` format and direct from text files. Once you’ve loaded the data into the `Dataset` object, you can string together various operations to apply to the data, these include operations such as:

* `batch()` – this allows you to consume the data from your TensorFlow Dataset in batches
* `map()` – this allows you to transform the data using lambda statements applied to each element
* `zip()` – this allows you to zip together different `Dataset` objects into a new Dataset, in a similar way to the Python zip function
* `filter()` – this allows you to remove problematic data-points in your data-set, again based on some lambda function
* `repeat()` – this operation restricts the number of times data is consumed from the Dataset before a `tf.errors.OutOfRangeError` error is thrown
* `shuffle()` – this operation shuffles the data in the Dataset

There are many other methods that the Dataset API includes – see https://www.tensorflow.org/api_docs/python/tf/data/Dataset for more details.  

The next component in the TensorFlow Dataset framework is the Iterator. This creates operations which can be called during the training, validation and/or testing of your model in TensorFlow. I’ll introduce more of both components in some examples below.

In [3]:
# First create a dataset out of numpy ranges
x = np.arange(0,10)

# Then we can create a TensorFlow Dataset object straight from the nparray using
# from_tensor_slices():
dx = tf.data.Dataset.from_tensor_slices(x)

* The object `dx `is now a TensorFlow `Dataset` object. 
* The next step is to create an Iterator that will extract data from this dataset. 
* In the code below, the iterator is created using the method `make_one_shot_iterator()`.  The iterator arising from this method can only be initialized and run once – it can’t be re-initialized. The importance of being able to re-initialize an iterator will be explained more later.

In [4]:
# create a one-shot iterator
iterator = dx.make_one_shot_iterator()
# extract an element
next_element = iterator.get_next()

In [7]:
with tf.Session() as sess:
    print(sess.run(next_element))
    print(sess.run(next_element))

0
1


In [5]:
next_element

<tf.Tensor 'IteratorGetNext:0' shape=() dtype=int64>

In [6]:
iterator.get_next()

<tf.Tensor 'IteratorGetNext_1:0' shape=() dtype=int64>

In [8]:
# extracts all the data and then throws an OutOfRangeError at end
with tf.Session() as sess:
    for i in range(11):
        val = sess.run(next_element)
        print(val)

0
1
2
3
4
5
6
7
8
9


OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[]], output_types=[DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator)]]

Caused by op 'IteratorGetNext', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 112, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.5/asyncio/base_events.py", line 345, in run_forever


If we want to repeatedly extract data from a dataset, one way we can do it is to make the dataset re-initializable. We can do that by first adjusting the make_one_shot_iterator() line to the following:

In [9]:
x = np.arange(0,30)
dx = tf.data.Dataset.from_tensor_slices(x)
#iterator = dx.make_one_shot_iterator()
iterator = dx.make_initializable_iterator()
next_element = iterator.get_next()

In [10]:
with tf.Session() as sess:
    # now the first operation iterator.initializer gets the iterator
    # ready for action before running the next_element operation
    sess.run(iterator.initializer)
    for i in range(15):
        val = sess.run(next_element)
        print(val)
        # if we run out of data, re-initialize
        if i % 9 == 0 and i > 0:
            sess.run(iterator.initializer)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


#### the `.batch()` function

In [11]:
dx = tf.data.Dataset.from_tensor_slices(x).batch(3)

In [12]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(15):
        val = sess.run(next_element)
        print([val], i)
        if (i + 1) % (10 // 3) == 0 and i > 0:
            sess.run(iterator.initializer)

[0] 0
[1] 1
[2] 2
[0] 3
[1] 4
[2] 5
[0] 6
[1] 7
[2] 8
[0] 9
[1] 10
[2] 11
[0] 12
[1] 13
[2] 14


#### zip datasets together to pair input-output training/validation pairs of data
* In this example, the batching takes place appropriately whith the zipped together datasets (i.e. 3 items from dx, 3 items from dy.
* The re-initialization `if` statement on the last two lines can be shorted by replacing the dcomb dataset creation line as seen below where adding the `.repeat()` method without argument which means the dataset can be repeated indefinitely without throwing an OutOfRangeError

In [13]:
x = np.arange(0,10)
y = np.arange(1, 11)

def simple_zip_example(x, y):
    # create dataset objects from the arrays
    dx = tf.data.Dataset.from_tensor_slices(x)
    dy = tf.data.Dataset.from_tensor_slices(y)
    
    # Zip the two datasets together
    #dcomb = tf.data.Dataset.zip((dx, dy)).batch(3)
    dcomb = tf.data.Dataset.zip((dx, dy)).repeat().batch(3)
    iterator = dcomb.make_initializable_iterator()
    
    # extract an element
    next_element = iterator.get_next()
    
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        for i in range(15):
            val = sess.run(next_element)
            print(val)
#             if (i+1) % (10 // 3) == 0 and i > 0:
#                sess.run(iterator.initializer)
    

In [14]:
simple_zip_example(x,y)

(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([9, 0, 1]), array([10,  1,  2]))
(array([2, 3, 4]), array([3, 4, 5]))
(array([5, 6, 7]), array([6, 7, 8]))
(array([8, 9, 0]), array([ 9, 10,  1]))
(array([1, 2, 3]), array([2, 3, 4]))
(array([4, 5, 6]), array([5, 6, 7]))
(array([7, 8, 9]), array([ 8,  9, 10]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([9, 0, 1]), array([10,  1,  2]))
(array([2, 3, 4]), array([3, 4, 5]))


### MNIST Dataset example

In [15]:
from sklearn.datasets import load_digits

In [16]:
digits = load_digits(return_X_y=True)

In [17]:
train_images = digits[0][:int(len(digits[0]) * 0.8)]
train_labels = digits[1][:int(len(digits[0]) * 0.8)]
valid_images = digits[0][int(len(digits[0]) * 0.8):]
valid_labels = digits[1][int(len(digits[0]) * 0.8):]

In [18]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(train_images)
# apply a one-hot transform to each label for use in the NN
dy_train = tf.data.Dataset.from_tensor_slices(train_labels).map(lambda z: tf.one_hot(z, 10))
# zip the x and y training_data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(30)

In [19]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(valid_images)
dy_valid = tf.data.Dataset.from_tensor_slices(valid_labels).map(lambda z: tf.one_hot(z, 10))
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

* Now, we want to be able to extract data from either the train_dataset or the valid_dataset seamlessly. 
* This is important, as we don’t want to have to change how data flows through the neural network structure when all we want to do is just change the dataset the model is consuming. 
* To do this, we can use another way of creating the Iterator object – the `from_structure()` method. 
* This method creates a generic iterator object – all it needs is the data types of the data it will be outputting and the output data size/shape in order to be created. 
The code below uses this methodology:

In [20]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                          train_dataset.output_shapes)
next_element = iterator.get_next()

# Now we need operations which can be called during training or eval to initialize
# this generic iterator and "point it" to the desired dataset.
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [21]:
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 50)
    fc2 = tf.layers.dense(fc1, 50)
    fc2 = tf.layers.dropout(fc2)
    fc3 = tf.layers.dense(fc2, 10)
    return fc3

* Note that the next_element operation is handled directly in the model – in other words, it doesn’t need to be called explicitly during the training loop as will be seen below.
* Rather, whenever any of the operations following this point in the graph are called (i.e. the loss operation, the optimization operation etc.) the TensorFlow graph structure will know to run the next_element operation and extract the data from whichever dataset has been initialized into the iterator. 
* The next_element operation, because it is operating on the generic iterator which is defined by the shape of the train_dataset, is a tuple – the first element ([0]) will contain the MNIST images, while the second element ([1]) will contain the corresponding labels. Therefore, next_element[0] will extract the image data batch and send it into the neural network model (nn_model) as the input data.

In [35]:
logits = nn_model(next_element[0])

In [37]:
# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# get accuracy
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
init_op = tf.global_variables_initializer()

In [39]:
# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i,l,acc*100))
    # now setup the validation run
    valid_iters = 100
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,avg_acc))

Epoch: 0, loss: 882.779, training accuracy: 10.00%
Epoch: 50, loss: 39.402, training accuracy: 56.67%
Epoch: 100, loss: 10.410, training accuracy: 90.00%
Epoch: 150, loss: 8.496, training accuracy: 93.33%
Epoch: 200, loss: 8.124, training accuracy: 90.00%
Epoch: 250, loss: 3.650, training accuracy: 93.33%
Epoch: 300, loss: 2.060, training accuracy: 100.00%
Epoch: 350, loss: 2.678, training accuracy: 96.67%
Epoch: 400, loss: 3.847, training accuracy: 96.67%
Epoch: 450, loss: 1.941, training accuracy: 100.00%
Epoch: 500, loss: 2.426, training accuracy: 100.00%
Epoch: 550, loss: 0.170, training accuracy: 100.00%
Average validation set accuracy over 100 iterations is 88.20%


https://github.com/FrancescoSaverioZuppichini/Tensorflow-Dataset-Tutorial/blob/master/dataset_tutorial.ipynb

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[0.12020319 0.55918353]


In [3]:
# using two numpy arrays
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

(array([0.85340944, 0.30916813]), array([0.4400224]))


In [4]:
# using a tensor 
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

[0.92103696 0.14862382]


In [5]:
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

data = np.random.sample((100,2))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer, feed_dict={ x: data })
    print(sess.run(el))

[0.09896725 0.19451222]


In [6]:
# from generator
sequence = np.array([[1],[2,3],[3,4]])

def generator():
    for el in sequence:
        yield el

dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])
iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

[1.]


In [7]:
# initializable iterator to switch between data
EPOCHS = 10

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])
#     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

[array([1., 2.], dtype=float32), array([0.], dtype=float32)]


In [8]:
# Reinitializable iterator to switch between Datasets
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([0.97778559, 0.44981254]), array([0.78315483])]


In [9]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[0.84278051 0.3618442 ]
 [0.64999931 0.0216752 ]
 [0.74922154 0.10555502]
 [0.62837537 0.11433535]]


In [16]:
# REPEAT
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat()

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))
#     this will run forever
#     while True:
#         print(sess.run(el))    

[1]


In [17]:
# MAP
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.map(lambda x: x*2)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))
#     this will run forever
#         for _ in range(len(x)):
#             print(sess.run(el))

[2]


In [18]:
# SHUFFLE
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[1]
 [4]
 [3]
 [2]]


In [19]:
# how to pass the value to a model
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
x, y = iter.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Iter: 0, Loss: 0.3128
Iter: 1, Loss: 0.3014
Iter: 2, Loss: 0.2902
Iter: 3, Loss: 0.2794
Iter: 4, Loss: 0.2690
Iter: 5, Loss: 0.2589
Iter: 6, Loss: 0.2491
Iter: 7, Loss: 0.2397
Iter: 8, Loss: 0.2306
Iter: 9, Loss: 0.2219


In [20]:
# Wrapping all together -> Switch between train and test set
EPOCHS = 10
BATCH_SIZE = 16
# create a placeholder to dynamically switch between batch sizes
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()

# using two numpy arrays
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

n_batches = len(train_data[0]) // BATCH_SIZE

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

Training...
Iter: 0, Loss: 0.3647
Iter: 1, Loss: 0.2157
Iter: 2, Loss: 0.1583
Iter: 3, Loss: 0.1214
Iter: 4, Loss: 0.0941
Iter: 5, Loss: 0.0935
Iter: 6, Loss: 0.0861
Iter: 7, Loss: 0.0815
Iter: 8, Loss: 0.0804
Iter: 9, Loss: 0.0853
Test Loss: 0.087634
